In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

llm_config = {"model": "gpt-4o"}

In [2]:
from autogen import ConversableAgent

/Users/martinfabbri/Workspace/github/graph-llm-agents/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
onboarding_personal_information_agent = ConversableAgent(
    name="Onboarding Personal Information Agent",
    system_message="""You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's name and location.
    Do not ask for other information. Return 'TERMINATE' 
    when you have gathered all the information.""",
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [4]:
onboarding_topic_preference_agent = ConversableAgent(
    name="Onboarding Topic preference Agent",
    system_message="""You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's preferences on news topics.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.""",
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [5]:
customer_engagement_agent = ConversableAgent(
    name="Customer Engagement Agent",
    system_message="""You are a helpful customer service agent
    here to provide fun for the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.""",
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

In [6]:
customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

In [7]:
chats = [
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message": 
            "Hello, I'm here to help you get started with our product."
            "Could you tell me your name and location?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the customer information "
                             "into as JSON object only: "
                             "{'name': '', 'location': ''}",
        },
        "max_turns": 2,
        "clear_history" : True
    },
    {
        "sender": onboarding_topic_preference_agent,
        "recipient": customer_proxy_agent,
        "message": 
                "Great! Could you tell me what topics you are "
                "interested in reading about?",
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history" : False
    },
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Let's find something fun to read.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
    },
]

In [8]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding Personal Information Agent (to customer_proxy_agent):

Hello, I'm here to help you get started with our product.Could you tell me your name and location?

--------------------------------------------------------------------------------


/Users/martinfabbri/Workspace/github/graph-llm-agents/.venv/lib/python3.12/site-packages/autogen/agentchat/chat.py:47: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(


customer_proxy_agent (to Onboarding Personal Information Agent):

Martin

--------------------------------------------------------------------------------
Onboarding Personal Information Agent (to customer_proxy_agent):

Thank you, Martin. Where are you located?

--------------------------------------------------------------------------------
customer_proxy_agent (to Onboarding Personal Information Agent):

Eugene, OR

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding Topic preference Agent (to customer_proxy_agent):

Great! Could you tell me what topics you are interested in reading about?
Context: 
{
  "name": "Martin",
  "location": "Eugene, OR"
}

--------------------------------------------------------------------------------
customer_proxy_agent (to Onbo

In [9]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

{
  "name": "Martin",
  "location": "Eugene, OR"
}


Martin, from Eugene, OR, is interested in reading about dogs.


Martin, from Eugene, OR, has an interest in dogs and enjoyed reading the heartwarming story of Hachiko, an Akita dog known for his unwavering loyalty in awaiting his owner daily at Shibuya Train Station even after the owner's unexpected death. This story highlights the incredible bond between humans and dogs, symbolized by a statue of Hachiko at Shibuya Station.




In [10]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")

{'usage_including_cached_inference': {'total_cost': 0.001325, 'gpt-4o-2024-05-13': {'cost': 0.001325, 'prompt_tokens': 175, 'completion_tokens': 30, 'total_tokens': 205}}, 'usage_excluding_cached_inference': {'total_cost': 0.001325, 'gpt-4o-2024-05-13': {'cost': 0.001325, 'prompt_tokens': 175, 'completion_tokens': 30, 'total_tokens': 205}}}


{'usage_including_cached_inference': {'total_cost': 0.0005600000000000001, 'gpt-4o-2024-05-13': {'cost': 0.0005600000000000001, 'prompt_tokens': 70, 'completion_tokens': 14, 'total_tokens': 84}}, 'usage_excluding_cached_inference': {'total_cost': 0.0005600000000000001, 'gpt-4o-2024-05-13': {'cost': 0.0005600000000000001, 'prompt_tokens': 70, 'completion_tokens': 14, 'total_tokens': 84}}}


{'usage_including_cached_inference': {'total_cost': 0.00836, 'gpt-4o-2024-05-13': {'cost': 0.00836, 'prompt_tokens': 505, 'completion_tokens': 389, 'total_tokens': 894}}, 'usage_excluding_cached_inference': {'total_cost': 0.00836, 'gpt-4o-2024-05-13': {'cost': 0